# Load R libraries

In [ ]:
set.seed(222)
library(monocle3)
library(ggplot2)
library(RColorBrewer)
library(pheatmap)
library(enrichR)
websiteLive <- getOption("enrichR.live")
dbs <- c("GO_Molecular_Function_2023", "GO_Cellular_Component_2023", "GO_Biological_Process_2023",
            "KEGG_2021_Human", "Elsevier_Pathway_Collection", "GWAS_Catalog_2023")
library(dplyr)
library(ggridges)

# Load R functions

## Initial settings: color, order, trait

In [ ]:
data_dir='Pseudotime_trajectories'
setwd(data_dir)
source("initial_settings.r")

## Load trajectory reconstruction and downstream analysis based on Kriegstein scripts

In [ ]:
source("trajectory_reconstruction.r")

## Load defined functions

In [ ]:
source("functions.r")

# traDEGs (q_value < 0.05 & Moran >= 0.05)

## Astrocyte

### Load data

In [ ]:
cds_AST <- readRDS("integrated_AST_con_afterCombination_cds_compressed.RDS") # This file can be download from Synapse: syn62283908
cds_AST
names(cds_AST@lineages)

### Merge DEGs

In [ ]:
DEGs_AST <- c()
for(sub_trajectory in names(cds_AST@lineages)){
    data <- read.table(paste0("files/traDEGs/AST_traDEG_", sub_trajectory,".txt"))
    DEGs_AST <- c(DEGs_AST, rownames(data[data$q_value < 0.05 & data$morans_I >= 0.05, ]))
}
DEGs_AST <- unique(DEGs_AST)
length(DEGs_AST)

### Obtain fitted matrix

In [ ]:
cds_AST_scale_start <- adjust_fitted_matrix(cds_AST)
merged_AST_scale_start <- t(rbind(cds_AST_scale_start@expectation[[1]][nrow(cds_AST_scale_start@expectation[[1]]):1, ], cds_AST_scale_start@expectation[[2]]))
merged_AST_scale_start <- merged_AST_scale_start[DEGs_AST, ]
merged_AST_scale_start

### 6 groups (*used*)

In [ ]:
ks_6_AST_term_3 <- cluster_and_func_enrich(fitted_mat = merged_AST_scale_start, n_tree = 6, 
                                        pseudo_range = sapply(cds_AST_scale_start@pseudotime, "max"), 
                                        lineages = names(cds_AST@lineages), 
                                        branches = c(rep("PA", 500), rep("FA", 500)), 
                                        n_metacell = 500, n_term = 3, n_char = 100, 
                                        out_label = "files/traDEGs/qvalue0.05_moran0.05/AST")

#### MAGMA

In [ ]:
magmasets <- readr::read_tsv("files/traDEGs/qvalue0.05_moran0.05/MAGMA_AST_ks_6/meta-files/geneSetResults.tsv.gz")
ngenes = lapply(unique(magmasets$VARIABLE), function(x) round(mean(magmasets[magmasets$VARIABLE==x,"NGENES"])))
names(ngenes) = unique(magmasets$VARIABLE)

magmasets = magmasets[(magmasets$gwasAcronym %in% names(trait_info)),]
magmasets = magmasets[order(magmasets$gwasAcronym),]
magmasets$sumstatName = magmasets$gwasAcronym
magmasets$sumstatName = ordered(magmasets$sumstatName, levels=rev(names(trait_info)))
magmasets$annoName = factor(magmasets$VARIABLE, levels = sort(unique(magmasets$VARIABLE)))
magmasets$tempScoreCol = magmasets$LogP

pdf("files/traDEGs/qvalue0.05_moran0.05/ast_degs_ks_6_magma.pdf", width = 7.5, height = 7.5)
magma_heatmap(magmasets, plotCol="LogP", markNominallySignificant=T)
dev.off()

## Microglia

### Load data

In [ ]:
cds_MICRO <- readRDS("integrated_MICRO_con_afterCombination_cds_compressed.RDS") # This file can be download from Synapse: syn62283907
cds_MICRO
names(cds_MICRO@lineages)

### Merge DEGs

In [ ]:
DEGs_MICRO <- c()
for(sub_trajectory in names(cds_MICRO@lineages)){
    data <- read.table(paste0("files/traDEGs/MICRO_traDEG_", sub_trajectory,".txt"))
    DEGs_MICRO <- c(DEGs_MICRO, rownames(data[data$q_value < 0.05 & data$morans_I >= 0.05, ]))
}
DEGs_MICRO <- unique(DEGs_MICRO)
length(DEGs_MICRO)

### Obtain fitted matrix

In [ ]:
cds_MICRO_scale_start <- adjust_fitted_matrix(cds_MICRO)
merged_MICRO_scale_start <- t(do.call(rbind, cds_MICRO_scale_start@expectation))
merged_MICRO_scale_start <- merged_MICRO_scale_start[DEGs_MICRO, ]
merged_MICRO_scale_start

### 4 groups

In [ ]:
ks_4_MICRO_term_3 <- cluster_and_func_enrich(fitted_mat = merged_MICRO_scale_start, n_tree = 4, 
                                        pseudo_range = sapply(cds_MICRO_scale_start@pseudotime, "max"), 
                                        lineages = names(cds_MICRO@lineages), 
                                        branches = rep("Micro", 500), 
                                        n_metacell = 500, n_term = 3, n_char = 100, 
                                        out_label = "files/traDEGs/qvalue0.05_moran0.05/MICRO")

#### MAGMA

In [ ]:
magmasets <- readr::read_tsv("files/traDEGs/qvalue0.05_moran0.05/MAGMA_MICRO_ks_4/meta-files/geneSetResults.tsv.gz")
ngenes = lapply(unique(magmasets$VARIABLE), function(x) round(mean(magmasets[magmasets$VARIABLE==x,"NGENES"])))
names(ngenes) = unique(magmasets$VARIABLE)

magmasets = magmasets[(magmasets$gwasAcronym %in% names(trait_info)),]
magmasets = magmasets[order(magmasets$gwasAcronym),]
magmasets$sumstatName = magmasets$gwasAcronym
magmasets$sumstatName = ordered(magmasets$sumstatName, levels=rev(names(trait_info)))
magmasets$annoName = factor(magmasets$VARIABLE, levels = sort(unique(magmasets$VARIABLE)))
magmasets$tempScoreCol = magmasets$LogP

pdf("files/traDEGs/qvalue0.05_moran0.05/micro_degs_ks_4_magma.pdf", width = 7.5, height = 7.5)
magma_heatmap(magmasets, plotCol="LogP", markNominallySignificant=T)
dev.off()

## Oligodendrocyte

### Load data

In [ ]:
cds_OLIGO <- readRDS("integrated_OLIGO_con_afterCombination_cds_compressed.RDS") # This file can be download from Synapse: syn62283899
cds_OLIGO
names(cds_OLIGO@lineages)

### Merge DEGs

In [ ]:
DEGs_OLIGO <- c()
for(sub_trajectory in names(cds_OLIGO@lineages)){
    data <- read.table(paste0("files/traDEGs/OLIGO_traDEG_", sub_trajectory,".txt"))
    DEGs_OLIGO <- c(DEGs_OLIGO, rownames(data[data$q_value < 0.05 & data$morans_I >= 0.05, ]))
}
DEGs_OLIGO <- unique(DEGs_OLIGO)
length(DEGs_OLIGO)

### Obtain fitted matrix

In [ ]:
cds_OLIGO_scale_start <- adjust_fitted_matrix(cds_OLIGO)
merged_OLIGO_scale_start <- t(do.call(rbind, cds_OLIGO_scale_start@expectation))
merged_OLIGO_scale_start <- merged_OLIGO_scale_start[DEGs_OLIGO, ]
merged_OLIGO_scale_start

### 2 groups

In [ ]:
ks_2_OLIGO_term_3 <- cluster_and_func_enrich(fitted_mat = merged_OLIGO_scale_start, n_tree = 2, 
                                        pseudo_range = sapply(cds_OLIGO_scale_start@pseudotime, "max"), 
                                        lineages = names(cds_OLIGO@lineages), 
                                        branches = rep("Oligo", 500), 
                                        n_metacell = 500, n_term = 3, n_char = 100, 
                                        out_label = "files/traDEGs/qvalue0.05_moran0.05/OLIGO")

#### MAGMA

In [ ]:
magmasets <- readr::read_tsv("files/traDEGs/qvalue0.05_moran0.05/MAGMA_OLIGO_ks_2/meta-files/geneSetResults.tsv.gz")
ngenes = lapply(unique(magmasets$VARIABLE), function(x) round(mean(magmasets[magmasets$VARIABLE==x,"NGENES"])))
names(ngenes) = unique(magmasets$VARIABLE)

magmasets = magmasets[(magmasets$gwasAcronym %in% names(trait_info)),]
magmasets = magmasets[order(magmasets$gwasAcronym),]
magmasets$sumstatName = magmasets$gwasAcronym
magmasets$sumstatName = ordered(magmasets$sumstatName, levels=rev(names(trait_info)))
magmasets$annoName = factor(magmasets$VARIABLE, levels = sort(unique(magmasets$VARIABLE)))
magmasets$tempScoreCol = magmasets$LogP

pdf("files/traDEGs/qvalue0.05_moran0.05/oligo_degs_ks_2_magma.pdf", width = 7.5, height = 7.5)
magma_heatmap(magmasets, plotCol="LogP", markNominallySignificant=T)
dev.off()

## Excitatory neuron

### Load data

In [ ]:
cds_EN <- readRDS("integrated_EN_con_afterCombination_cds_compressed.RDS") # This file can be download from Synapse: syn62283906
cds_EN

In [ ]:
names(cds_EN@lineages)

### Merge DEGs

In [ ]:
DEGs_EN <- c()
for(sub_trajectory in names(cds_EN@lineages)){
    data <- read.table(paste0("files/traDEGs/EN_traDEG_", sub_trajectory,".txt"))
    DEGs_EN <- c(DEGs_EN, rownames(data[data$q_value < 0.05 & data$morans_I >= 0.05, ]))
}
DEGs_EN <- unique(DEGs_EN)
length(DEGs_EN)

### Obtain fitted matrix

In [ ]:
cds_EN_scale_start <- adjust_fitted_matrix(cds_EN)
merged_EN_scale_start <- t(do.call(rbind, cds_EN_scale_start@expectation))
merged_EN_scale_start <- merged_EN_scale_start[DEGs_EN, ]
merged_EN_scale_start

### 8 groups

In [ ]:
ks_8_EN_term_3 <- cluster_and_func_enrich(fitted_mat = merged_EN_scale_start, n_tree = 8, 
                                          pseudo_range = sapply(cds_EN_scale_start@pseudotime, "max"), 
                                          lineages = names(cds_EN@lineages), 
                                          branches = c(rep("Deep-non-IT", 1500), rep("Deep-IT", 1000), rep("Upper-IT", 2000)), 
                                          n_metacell = 500, n_term = 3, n_char = 100, 
                                          out_label = "files/traDEGs/qvalue0.05_moran0.05/EN", 
                                          width = 10, 
                                          height = 12)

#### MAGMA

In [ ]:
magmasets <- readr::read_tsv("files/traDEGs/qvalue0.05_moran0.05/MAGMA_EN_ks_8/meta-files/geneSetResults.tsv.gz")
ngenes = lapply(unique(magmasets$VARIABLE), function(x) round(mean(magmasets[magmasets$VARIABLE==x,"NGENES"])))
names(ngenes) = unique(magmasets$VARIABLE)

magmasets = magmasets[(magmasets$gwasAcronym %in% names(trait_info)),]
magmasets = magmasets[order(magmasets$gwasAcronym),]
magmasets$sumstatName = magmasets$gwasAcronym
magmasets$sumstatName = ordered(magmasets$sumstatName, levels=rev(names(trait_info)))
magmasets$annoName = factor(magmasets$VARIABLE, levels = sort(unique(magmasets$VARIABLE)))
magmasets$tempScoreCol = magmasets$LogP

pdf("files/traDEGs/qvalue0.05_moran0.05/en_degs_ks_8_magma.pdf", width = 7.5, height = 7.5)
magma_heatmap(magmasets, plotCol="LogP", markNominallySignificant=T)
dev.off()

## Inhibitory neuron

### Load data

In [ ]:
cds_IN <- readRDS("integrated_IN_con_afterCombination_cds_compressed.RDS") # This file can be download from Synapse: syn62283892
cds_IN
names(cds_IN@lineages)

### Merge DEGs

In [ ]:
DEGs_IN <- c()
for(sub_trajectory in names(cds_IN@lineages)){
    data <- read.table(paste0("files/traDEGs/IN_traDEG_", sub_trajectory,".txt"))
    DEGs_IN <- c(DEGs_IN, rownames(data[data$q_value < 0.05 & data$morans_I >= 0.05, ]))
}
DEGs_IN <- unique(DEGs_IN)
length(DEGs_IN)

### Obtain fitted matrix

In [ ]:
cds_IN_scale_start <- adjust_fitted_matrix(cds_IN)
merged_IN_scale_start <- t(do.call(rbind, cds_IN_scale_start@expectation))
merged_IN_scale_start <- merged_IN_scale_start[DEGs_IN, ]
merged_IN_scale_start

### 8 groups

In [ ]:
ks_8_IN_term_3 <- cluster_and_func_enrich(fitted_mat = merged_IN_scale_start, n_tree = 8, 
                                          pseudo_range = sapply(cds_IN_scale_start@pseudotime, "max"), 
                                          lineages = names(cds_IN@lineages), 
                                          branches = c(rep("MGE", 1500), rep("CGE", 4000)), 
                                          n_metacell = 500, n_term = 3, n_char = 100, 
                                          out_label = "files/traDEGs/qvalue0.05_moran0.05/IN", 
                                          width = 30, 
                                          height = 12)

In [ ]:
names(cds_IN@lineages)

#### MAGMA

In [ ]:
magmasets <- readr::read_tsv("files/traDEGs/qvalue0.05_moran0.05/MAGMA_IN_ks_8/meta-files/geneSetResults.tsv.gz")
ngenes = lapply(unique(magmasets$VARIABLE), function(x) round(mean(magmasets[magmasets$VARIABLE==x,"NGENES"])))
names(ngenes) = unique(magmasets$VARIABLE)

magmasets = magmasets[(magmasets$gwasAcronym %in% names(trait_info)),]
magmasets = magmasets[order(magmasets$gwasAcronym),]
magmasets$sumstatName = magmasets$gwasAcronym
magmasets$sumstatName = ordered(magmasets$sumstatName, levels=rev(names(trait_info)))
magmasets$annoName = factor(magmasets$VARIABLE, levels = sort(unique(magmasets$VARIABLE)))
magmasets$tempScoreCol = magmasets$LogP

pdf("files/traDEGs/qvalue0.05_moran0.05/in_degs_ks_8_magma.pdf", width = 7.5, height = 7.5)
magma_heatmap(magmasets, plotCol="LogP", markNominallySignificant=T)
dev.off()